# Notebook for Running Biomechanics Simulations with Reboot Motion Data

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples_biomechanics/RebootMotionBiomechanicsSimulation.ipynb)__

Run the cells in order, making sure to enter AWS credentials in the cell when prompted

In [ ]:
#@title Install Python Package

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@jacob/mujoco-batch-processing
!pip install git+https://github.com/RebootMotion/mlb-statsapi.git@v1.1.0#egg=mlb_statsapi > /dev/null
!echo "Done Installing Reboot Toolkit"

In [ ]:
#@title Install MuJoCo Viewer

!pip install -q mjc_viewer
print("Done installing Mujoco Viewer")

In [ ]:
#@title Import Python Libraries

import IPython
import matplotlib.pyplot as plt
import mujoco
import numpy as np
import os
import pandas as pd
from random import choice
import datetime

from IPython.display import display
from tqdm import tqdm

import reboot_toolkit as rtk

from reboot_toolkit import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata, setup_aws, decorate_primary_segment_df_with_stats_api

In [ ]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
# OR input your credentials string generated by the Reboot Dashboard

boto3_session = setup_aws()

In [ ]:
# The roster file is a csv file with 4 required columns
# 'Name': the name of the pitcher
# 'ID': The Org ID for each pitcher
# 'weight': The weight of each pitcher in kilograms
# 'hand': The throwing hand of each pitcher

roster_filename = ' .csv'

In [ ]:
roster = pd.read_csv(roster_filename)
roster

In [ ]:
#@title User Input - No code changes required below this section, just enter information in forms

# See https://docs.rebootmotion.com/ for all available file types and the data in each
mocap_types = [MocapType.HAWKEYE_HFR, MocapType.HAWKEYE]
movement_type = MovementType.BASEBALL_PITCHING
handedness = Handedness.RIGHT
file_type = FileType.INVERSE_KINEMATICS

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'

# Use this bool to add columns of data, like pitch_type and start_speed, from the stats API
add_stats_api = False  # True or False

if add_stats_api:
    print("Will add data from the Stats API like velo and pitch type")

else:
    print("Will NOT add data from the Stats API like velo and pitch type (set to True above if needed)")

In [ ]:
#@title The Function for Dynamically Simulating a Single Play with MuJoCo

from mjc_viewer import Serializer, Trajectory

def mujoco_sim(mj_model, mj_joint_names, positions_df, do_render=False):

    # set the suffix to be used for all resulting simulation columns
    col_suffix = "invdyn"

    # create the mujoco data element for the simulation
    mj_data = mujoco.MjData(mj_model)

    # set the simulation time step as the median time step in the play (all should be uniform)
    dt = positions_df['time'].diff().median()
    mj_model.opt.timestep = dt

    # reorder the IK data frame to make it easier to set the values in the simulation
    positions_df = rtk.reorder_joint_angle_df_like_model(
        mj_model, mj_data, positions_df.copy(), mj_joint_names
    )
    angle_cols = [
        col for col in list(positions_df) if not col.endswith('translation')
    ]
    positions_df[angle_cols] = positions_df[angle_cols].apply(np.radians)

    # compute the gradient of the positions
    # (the dynamic state of the model includes both positions and velocities)
    velocities_df = positions_df.copy().apply(np.gradient, raw=True) / dt

    sim_results = []

    if do_render:
        # create a Serializer and Trajectory instance
        serializer = Serializer(mj_model)
        trajectory = Trajectory(mj_data)

    else:
        trajectory = None
        serializer = None

    # Below is the most straightforward simulation loop with MuJoCo using "mj_step"
    # Here is documentation for the general simulation framework:
    # https://mujoco.readthedocs.io/en/latest/programming/simulation.html
    for i, row_pos in positions_df.iterrows():

        row_vel = velocities_df.iloc[i]

        mj_data.qpos = row_pos.to_numpy()

        mj_data.qvel = row_vel.to_numpy()

        mujoco.mj_step(mj_model, mj_data)

        mujoco.mj_inverse(mj_model, mj_data)

        if trajectory is not None:
            # store results in trajectory instance
            trajectory.step()

        # here you can save any simulation values you'd like...
        # see here for options: https://mujoco.readthedocs.io/en/stable/APIreference/APItypes.html#mjdata
        sim_results.append(mj_data.qfrc_inverse.copy())

    sim_results_df = pd.DataFrame(
        data=sim_results, columns=[f"{jn}_{col_suffix}" for jn in mj_joint_names]
    )

    if trajectory is not None:
        return sim_results_df, serializer, trajectory

    return sim_results_df

In [ ]:
#@title The Function for Running a Biomechanics Simulation for Each Play in a Data Segment

def simulate_segment_df(model_xml_str, segment_data_df):

    model = mujoco.MjModel.from_xml_string(model_xml_str)

    joint_names = rtk.get_model_info(model_xml_str, 'joint', return_names=True)

    sim_dfs = []

    print('Running a simulation for each play in the data segment...')
    for org_movement_id in tqdm(segment_data_df['org_movement_id'].unique()):

        ik_df = segment_data_df.loc[
            segment_data_df['org_movement_id'] == org_movement_id
        ].copy().reset_index(drop=True)

        current_sim_df = mujoco_sim(model, joint_names, ik_df)

        sim_dfs.append(current_sim_df)

    sim_df = pd.concat(sim_dfs, ignore_index=True)

    return pd.concat([segment_data_df, sim_df], axis=1)

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=[MocapType.HAWKEYE_HFR, MocapType.HAWKEYE],
    movement_type=MovementType.BASEBALL_PITCHING,
    handedness=Handedness.RIGHT,
    file_type=FileType.INVERSE_KINEMATICS,
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Display the Interface for Selecting the Primary Data Segment to Analyze

# Run this cell to display the dropdown menus and reset all options to NULL
# Note: Only the date range and year information will be used
primary_segment_widget = rtk.create_interactive_widget(s3_df)
display(primary_segment_widget)

In [ ]:
if not os.path.exists('player_data'):
    os.mkdir('player_data')

In [ ]:
roster_result_values = {}
for i in roster.index:
    primary_segment_data = primary_segment_widget.children[1].result
    player_meta_data = PlayerMetadata(
        org_player_ids=[roster.loc[i,'ID'].astype(str)],
        session_dates=None,
        session_nums=None,
        session_date_start=primary_segment_data["session_date_start"],
        session_date_end=primary_segment_data["session_date_end"],
        year=primary_segment_data["year"],
        org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
        s3_metadata=s3_metadata,
    )

    primary_segment_summary_df = rtk.filter_s3_summary_df(player_meta_data, s3_df)

    primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(
        primary_segment_summary_df['s3_path_delivery'].tolist(), add_ik_joints=True, add_elbow_var_val=True
    )

    # Set the mass for the player in the primary segment (use kilograms)
    primary_player_mass = roster.loc[i,'weight']

    player_handedness = roster.loc[i,'hand']

    # Retrieve the scaled model (both in length and mass) from AWS
    primary_model_xml_str = rtk.scale_human_xml(primary_segment_data_df, primary_player_mass, boto3_session=boto3_session)

    # Simulate all the plays in the primary segment with the retrieved model
    primary_sim_data_df = simulate_segment_df(primary_model_xml_str, primary_segment_data_df)

    # Filter dataframe
    filtered_df = primary_sim_data_df[(primary_sim_data_df['time_from_max_hand'] >= -0.2) & (primary_sim_data_df['time_from_max_hand'] <= 0.05)]

    # Group by and aggregate
    result = filtered_df.groupby(['session_num', 'org_movement_id']).agg({
        f'{player_handedness}_shoulder_rot_invdyn': ['min', 'max'],
        f'{player_handedness}_elbow_var_invdyn': ['min', 'max'],
        f'{player_handedness}_elbow_invdyn': ['min', 'max']
    }).reset_index()

    # Flatten MultiIndex and rename columns
    result.columns = ['_'.join(col).strip() if col[1] else col[0] for col in result.columns.values]
    result.rename(columns={
        'right_shoulder_rot_invdyn_min': 'shoulder_rot_invdyn_min',
        'right_shoulder_rot_invdyn_max': 'shoulder_rot_invdyn_max',
        'right_elbow_var_invdyn_min': 'elbow_var_invdyn_min',
        'right_elbow_var_invdyn_max': 'elbow_var_invdyn_max',
        'right_elbow_invdyn_min': 'elbow_invdyn_min',
        'right_elbow_invdyn_max': 'elbow_invdyn_max',
        'left_shoulder_rot_invdyn_min': 'shoulder_rot_invdyn_min',
        'left_shoulder_rot_invdyn_max': 'shoulder_rot_invdyn_max',
        'left_elbow_var_invdyn_min': 'elbow_var_invdyn_min',
        'left_elbow_var_invdyn_max': 'elbow_var_invdyn_max',
        'left_elbow_invdyn_min': 'elbow_invdyn_min',
        'left_elbow_invdyn_max': 'elbow_invdyn_max'
    }, inplace=True)

    player_season_mean = result.median(numeric_only=True).round(1)
    player_season_std = result.std(numeric_only=True).round(1)
    player_game_avg = result.drop(columns='org_movement_id').groupby('session_num').agg(['median', 'std']).round(1)

    player_game_avg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in player_game_avg.columns.values]

    player_result_values = {}
    for key in player_season_mean.keys():
        player_result_values[f'{key}_median'] = player_season_mean[key]
        player_result_values[f'{key}_std'] = player_season_std[key]

    player_name = roster.loc[i,'Name']

    roster_result_values[player_name] = player_result_values

    player_game_avg.to_csv(f'player_data/{player_name}.csv')



In [ ]:
roster_result_values = pd.DataFrame.from_dict(roster_result_values,orient='index')

In [ ]:
roster_result_values.to_csv('roster-results.csv')